In [17]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from os import listdir
from os.path import isfile, join
import requests

In [29]:
# load csv files collected from data-world
raleigh_crime = "Resources/RaleighPoliceIncidents.csv"
durham_crime = "Resources/city-of-durham-police-crime-reports_4.xlsx"
ch_crime = "Resources/unc-police-data.csv"
cary_crime = "Resources/crime-mapping_jr.csv"

In [23]:
#put csv files into dataframes
df_raleigh_crime = pd.read_csv(raleigh_crime)
df_raleigh_crime.head()

,OBJECTID,GlobalID,case_number,crime_category,crime_code,crime_description,crime_type,reported_block_address,city_of_incident,city,...,reported_date,reported_year,reported_month,reported_day,reported_hour,reported_dayofwk,latitude,longitude,agency,updated_date
0,12001,9cdee08d-11c8-4789-864b-6965a1b2e620,NaN,MISCELLANEOUS,81H,Miscellaneous/Missing Person (18 & over),NaN,NaN,NaN,RALEIGH,...,2017-01-15T03:28:00.000Z,2017,1,14,22,Saturday,NaN,NaN,RPD,2017-01-19T20:11:28.000Z
1,12002,6f6731f4-dd64-44c7-895c-555de2703c8a,NaN,MISCELLANEOUS,81A,Miscellaneous/All Other Non-Offenses,NaN,NaN,NaN,RALEIGH,...,2017-07-29T12:35:00.000Z,2017,7,29,8,Saturday,NaN,NaN,RPD,2017-08-01T14:06:50.000Z
2,12003,f0fd0e92-448e-4ca8-86c9-e6594564318b,NaN,MISCELLANEOUS,81F,Miscellaneous/Mental Commitment,NaN,NaN,NaN,RALEIGH,...,2016-03-07T03:52:00.000Z,2016,3,6,22,Sunday,NaN,NaN,RPD,2016-04-14T14:43:38.000Z
3,12004,8a212e84-7b53-478a-b225-c212aa25d7fd,NaN,MISCELLANEOUS,81A,Miscellaneous/All Other Non-Offenses,NaN,NaN,NaN,RALEIGH,...,2015-03-24T04:59:00.000Z,2015,3,24,0,Tuesday,NaN,NaN,RPD,2015-03-25T19:24:28.000Z
4,12005,01614b98-48f5-4374-a561-17c4b29d8857,NaN,MISCELLANEOUS,81A,Miscellaneous/All Other Non-Offenses,NaN,NaN,NaN,RALEIGH,...,2015-12-23T00:57:00.000Z,2015,12,22,19,Tuesday,NaN,NaN,RPD,2016-01-13T19:29:51.000Z


In [26]:
df_durham_crime = pd.read_excel(durham_crime) 
df_durham_crime.head()

,geo_point_2d,inci_id,date_rept,hour_rept,yearstamp,monthstamp,date_occu,dow1,hour_occu,date_fnd,...,chrgdesc,csstatus,csstatusdt,addtime,reviewdate,ucr_type_o,dist,strdate,big_zone,Unnamed: 23
0,35.949762,-78.996264,13000190,2013-01-03 00:00:00+00:00,921,2013,1,2013-01-02 00:00:00+00:00,WE,2300,...,650,LARCENY - AUTOMOBILE PARTS OR ACCESSORIES,1.0,2013-01-03 00:00:00+00:00,2013-01-03 00:00:00+00:00,2013-01-04 00:00:00+00:00,NaN,3.0,Jan 3 2013,Zone 3
1,35.981093,-78.890730,13000148,2013-01-02 00:00:00+00:00,1845,2013,1,2013-01-02 00:00:00+00:00,WE,1830,...,1400,VANDALISM TO AUTO (NOT ACCIDENTAL),1.0,2013-01-02 00:00:00+00:00,2013-01-02 00:00:00+00:00,2013-01-15 00:00:00+00:00,NaN,4.0,Jan 2 2013,Zone 4
2,35.987967,-78.893959,13000195,2013-01-03 00:00:00+00:00,958,2013,1,2012-12-24 00:00:00+00:00,MO,1200,...,1400,VANDALISM TO PROP (NOT ACCIDENTAL),2.0,2013-01-08 00:00:00+00:00,2013-01-03 00:00:00+00:00,2013-01-14 00:00:00+00:00,NaN,5.0,Jan 3 2013,Zone 4
3,36.005855,-78.952981,13000070,2013-01-01 00:00:00+00:00,1859,2013,1,2013-01-01 00:00:00+00:00,TU,1858,...,510,BURGLARY - FORCIBLE ENTRY,3.0,2013-01-01 00:00:00+00:00,2013-01-01 00:00:00+00:00,2013-01-14 00:00:00+00:00,NaN,3.0,Jan 1 2013,Zone 3
4,36.032530,-78.905660,13000160,2013-01-02 00:00:00+00:00,2129,2013,1,2013-01-02 00:00:00+00:00,WE,2129,...,4010,ALL TRAFFIC (EXCEPT DWI),3.0,2013-01-02 00:00:00+00:00,2013-01-02 00:00:00+00:00,2013-01-09 00:00:00+00:00,NaN,2.0,Jan 2 2013,Zone 2


In [27]:
df_ch_crime = pd.read_csv(ch_crime)
df_ch_crime.head()

,incident,date-time,year,date-no year,location,res hall,alcohol
0,NaN,1/1/13 0:00,2013,01/01,E FRANKLIN ST,NaN,NaN
1,EMS ASSIST,1/1/13 2:25,2013,01/01,AYCOCK CIRCLE PARKING LOT UNC,NaN,NaN
2,SUSPICIOUS CONDITION (NON-CRIMINAL),1/1/13 2:49,2013,01/01,S COLUMBIA ST/E CAMERON AVE,NaN,NaN
3,EMS ASSIST,1/1/14 12:25,2014,01/01,FINLEY CLUB HOUSE UNC,NaN,NaN
4,WELL-BEING CHECK,1/1/14 20:17,2014,01/01,CRAIGE RES HALL UNC,RES HALL,NaN


In [30]:
df_cary_crime = pd.read_csv(cary_crime)
df_cary_crime.head()

,crime_category,crime_type,chrgcnt,ucr,district,beat_number,map_reference,incident_number,time_from,time_to,...,phxrecordstatus,phxcommunity,phxstatus,record,offensecategory,violentproperty,domestic,location,radio,street
0,ALL OTHER,ALL OTHER - BLACKMAIL/ EXTORTION,NaN,2630.0,D1,114.0,P026,17000426.0,0001-01-01 03:40:00+00:00,0001-01-01 04:47:00+00:00,...,NaN,No,NaN,3011.0,All Other Offenses,All Other,N,35.788671414; -78.79625675,Everything,NaN
1,ALL OTHER,ALL OTHER - BLACKMAIL/ EXTORTION,NaN,2630.0,D3,118.0,P004,16009674.0,0001-01-01 11:53:00+00:00,0001-01-01 15:15:00+00:00,...,Suspende,Yes,Active,3019.0,All Other Offenses,All Other,N,35.741041415; -78.75461019,Everything,NaN
2,ALL OTHER,ALL OTHER - ESCAPE FROM CUSTODY OR RESIST ARREST,NaN,2650.0,D1,111.0,P024,17006197.0,0001-01-01 23:24:00+00:00,0001-01-01 23:24:00+00:00,...,NaN,No,NaN,3101.0,All Other Offenses,All Other,N,35.802211609; -78.78662602,Everything,NaN
3,ALL OTHER,ALL OTHER - ESCAPE FROM CUSTODY OR RESIST ARREST,NaN,2650.0,D1,113.0,P060,17008206.0,0001-01-01 14:45:00+00:00,0001-01-01 14:45:00+00:00,...,NaN,No,NaN,3198.0,All Other Offenses,All Other,N,35.788318078; -78.78256802,Everything,NaN
4,ALL OTHER,ALL OTHER - ESCAPE FROM CUSTODY OR RESIST ARREST,NaN,2650.0,D1,114.0,P026,16009915.0,0001-01-01 05:02:00+00:00,0001-01-01 05:02:00+00:00,...,NaN,No,NaN,3213.0,All Other Offenses,All Other,N,35.791369365; -78.78357141,Everything,NaN
